In [96]:
import os
import pickle
os.chdir("C:\Spring_2017\machine_learning\DS1003_Final_Project")
result = pickle.load( open( "result.p", "rb" ) )

In [103]:
import nltk
import numpy as np
import random
import pandas as pd
from nltk.corpus import wordnet
from nltk.corpus import semcor
from utilities import *

In [139]:
print(wordnet.synsets('enjoy'))
wordnet.synset('delight.v.02').definition()
wordnet.synset('enjoy.v.01').lemmas()
wordnet.synsets('relish')

[Synset('enjoy.v.01'), Synset('enjoy.v.02'), Synset('love.v.02'), Synset('enjoy.v.04'), Synset('delight.v.02')]


[Synset('gusto.n.01'),
 Synset('relish.n.02'),
 Synset('relish.n.03'),
 Synset('enjoy.v.01')]

In [6]:
sents_tag_pos = semcor.tagged_sents(tag='pos')
sents_tag_cor = semcor.tagged_sents(tag='sem')
sents_tag_both = semcor.tagged_sents(tag='semcor')

In [141]:
result = pickle.load( open( "result.p", "rb" ) )
Baseline_acc(result, 15)
#m_most_freq(result, 15)

{'act': 0.45588235294117646,
 'anode': 1.0,
 'be': 0.6049149338374291,
 'form': 0.7441860465116279,
 'have': 0.6261682242990654,
 'high': 0.6785714285714286,
 'information': 0.9836065573770492,
 'not': 1.0,
 'person': 1.0,
 'play': 0.22950819672131148,
 'report': 0.5892857142857143,
 'state': 0.5966386554621849,
 'stress': 0.9054054054054054,
 'sum': 0.7868852459016393,
 'use': 0.5662650602409639}

In [99]:
def find_pos(word):
    pos = {}
    for i in range(37176):
        #len(sents_tag_pos)
        #37176
        if i%10000==0:
            print(i)
        try:
            pos[i] = [x.label().synset().name().split(".")[0] if(len(str(x))>15) else x[0] for x in sents_tag_cor[i]].index(word)
        except:
            continue
    return pos


def getContextEmb(pos, window_size):
    cor_dict={}
    pos_dict={}
    tar_dict={}
    head_flag=False
    tail_flag=False
    for idx, center in pos.items():
        if idx%100==0:
            print(idx)
        # idx is the index of sentence that contain the word
        # center is the index of word in the sentence
        sent_len = len(sents_tag_cor[idx])
        start_pos = max([0,center-window_size])
        end_pos = min([len(sents_tag_cor[idx]),(center+window_size)+1])
        
        #cor_dict[idx] = [x.label().synset().name().split('.')[1] if(x.label() \
        #                 is not None and not isinstance(x.label(),str)) \
        #                 else "None" if(x.label() is None) \
        #                 else "NA" for x in sents_tag_both[idx][start_pos:end_pos]]
  
        pos_dict[idx] = [x.label() if(x.label() is not None) \
                         else "None" for x in sents_tag_pos[idx][start_pos:end_pos]]

        if center-window_size<0:
            pos_dict[idx] = ["start"]*(window_size-center) + pos_dict[idx]
                       
        if sent_len<(center+window_size)+1:
            pos_dict[idx] = pos_dict[idx] + ["end"]*((center+window_size)+1-sent_len)
              
        try:
            tar_dict[idx] = sents_tag_both[idx][center].label().synset().name()
        except:
            tar_dict[idx] = sents_tag_both[idx][center].label()
            
    return pos_dict, tar_dict

In [69]:
word_list = ['act','be','have','form','high','play','report','state','sum','use']

In [70]:
for word in word_list:
    print(word)
    pos = find_pos(word)
    pickle.dump(pos, open( word+".p", "wb" ) )
    pos_dict, tar_dict = getContextEmb(pos, 3)
    df1 = pd.DataFrame(pos_dict).transpose()
    pickle.dump(df1, open( "pos_" + word + ".p", "wb" ) )
    df3 = pd.Series(tar_dict)
    pickle.dump(df3, open( "tar_" + word + ".p", "wb" ))

act
0
10000
20000
30000
32000
be
0
10000
20000
30000
200
700
1000
1400
1600
1900
2000
2100
2200
2400
2800
3000
3600
4100
4200
4300
6100
6200
6900
7100
7400
8900
9600
9800
10100
10700
11700
12600
12900
13300
13700
14100
14600
14800
15300
15400
15600
15900
17800
18000
18400
19300
19400
19500
19600
22500
24400
24500
25300
26100
26200
26900
27700
28100
28300
29700
29900
30000
31200
31500
31800
32000
32800
33000
33100
33200
33500
33900
35900
36200
have
0
10000
20000
30000
5200
8200
10500
13200
14800
15400
16000
20000
26600
27200
27500
32800
33900
form
0
10000
20000
30000
15900
31900
high
0
10000
20000
30000
2000
14900
15700
play
0
10000
20000
30000
21700
36800
report
0
10000
20000
30000
11400
14800
34600
state
0
10000
20000
30000
0
100
400
600
5700
7000
8300
10400
10500
13200
14100
16700
19300
20000
20400
21100
33400
35300
36700
36800
sum
0
10000
20000
30000
15600
use
0
10000
20000
30000
3500
21300
29100


In [100]:
word_list = ['act','be','have','form','high','play','report','state','sum','use']
for word in word_list:
    print(word)
    pos = pickle.load(open(word+".p", "rb" ))
    pos_dict, tar_dict = getContextEmb(pos, 2)
    df1 = pd.DataFrame(pos_dict).transpose()
    pickle.dump(df1, open( "pos_" + word + "_2.p", "wb" ) )
    df3 = pd.Series(tar_dict)
    pickle.dump(df3, open( "tar_" + word + "_2.p", "wb" ))

act
32000
be
200
700
1000
1400
1600
1900
2000
2100
2200
2400
2800
3000
3600
4100
4200
4300
6100
6200
6900
7100
7400
8900
9600
9800
10100
10700
11700
12600
12900
13300
13700
14100
14600
14800
15300
15400
15600
15900
17800
18000
18400
19300
19400
19500
19600
22500
24400
24500
25300
26100
26200
26900
27700
28100
28300
29700
29900
30000
31200
31500
31800
32000
32800
33000
33100
33200
33500
33900
35900
36200
have
5200
8200
10500
13200
14800
15400
16000
20000
26600
27200
27500
32800
33900
form
15900
31900
high
2000
14900
15700
play
21700
36800
report
11400
14800
34600
state
0
100
400
600
5700
7000
8300
10400
10500
13200
14100
16700
19300
20000
20400
21100
33400
35300
36700
36800
sum
15600
use
3500
21300
29100
